
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Apache_Parquet_logo.svg" width=400 alt="Parquet Logo"></img>

# Store virtual datasets as Kerchunk Parquet references

## Overview
   
In this notebook we will cover how to store virtual datasets as Kerchunk Parquet references instead of Kerchunk JSON references. For large virtual datasets, using Parquet should have performance implications as the overall reference file size should be smaller and the memory overhead of combining the reference files should be lower. 


This notebook builds upon the [Kerchunk Basics](notebooks/foundations/01_kerchunk_basics.ipynb), [Multi-File Datasets with Kerchunk](notebooks/foundations/02_kerchunk_multi_file.ipynb) and the [Kerchunk and Dask](notebooks/foundations/03_kerchunk_dask.ipynb) notebooks. 

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Basics of virtual Zarr stores](../foundations/01_kerchunk_basics.ipynb) | Required | Core |
| [Multi-file virtual datasets with VirtualiZarr](../foundations/02_kerchunk_multi_file.ipynb) | Required | Core |
| [Parallel virtual dataset creation with VirtualiZarr, Kerchunk, and Dask](../foundations/03_kerchunk_dask) | Required | Core |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Required | IO/Visualization |

- **Time to learn**: 30 minutes
---

## Imports




In [ ]:
import logging

import dask
import fsspec
import xarray as xr
from distributed import Client
from virtualizarr import open_virtual_dataset

## Setting up the `Dask` Client


In [ ]:
client = Client(n_workers=8, silence_logs=logging.ERROR)
client

## Create Input File List

Here we are using `fsspec's` glob functionality along with the *`*`* wildcard operator and some string slicing to grab a list of NetCDF files from a `s3` `fsspec` filesystem. 

In [ ]:
# Initiate fsspec filesystems for reading
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)

files_paths = fs_read.glob("s3://smn-ar-wrf/DATA/WRF/DET/2022/12/31/12/*")

# Here we prepend the prefix 's3://', which points to AWS.
files_paths = sorted(["s3://" + f for f in files_paths])

### Subset the Data
To speed up our example, lets take a subset of the year of data. 

In [ ]:
# If the subset_flag == True (default), the list of input files will
# be subset to speed up the processing
subset_flag = True
if subset_flag:
    files_paths = files_paths[0:4]

# Generate Lazy References

Here we create a function to generate a list of Dask delayed objects.

In [ ]:
def generate_virtual_dataset(file, storage_options):
    return open_virtual_dataset(
        file, indexes={}, reader_options={"storage_options": storage_options}
    )


storage_options = dict(anon=True, default_fill_cache=False, default_cache_type="first")
# Generate Dask Delayed objects
tasks = [
    dask.delayed(generate_virtual_dataset)(file, storage_options)
    for file in files_paths
]

### Start the Dask Processing
To view the processing you can view it in real-time on the Dask Dashboard. ex: http://127.0.0.1:8787/status

In [ ]:
virtual_datasets = list(dask.compute(*tasks))

## Combine virtual datasets using VirtualiZarr

In [ ]:
combined_vds = xr.combine_nested(
    virtual_datasets, concat_dim=["time"], coords="minimal", compat="override"
)
combined_vds

## Write the virtual dataset to a Kerchunk Parquet reference

In [ ]:
combined_vds.virtualize.to_kerchunk("combined.parq", format="parquet")

# Shutdown the Dask cluster

In [ ]:
client.shutdown()

## Load kerchunked dataset

Next we initiate a `fsspec` `ReferenceFileSystem`.
We need to pass:
- The name of the parquet store
- The remote protocol (This is the protocol of the input file urls)
- The target protocol (`file` since we saved our parquet store locally).

In [ ]:
storage_options = {
    "remote_protocol": "s3",
    "skip_instance_cache": True,
    "remote_options": {"anon": True},
    "target_protocol": "file",
    "lazy": True,
}  # options passed to fsspec
open_dataset_options = {"chunks": {}}  # opens passed to xarray

ds = xr.open_dataset(
    "combined.parq",
    engine="kerchunk",
    storage_options=storage_options,
    open_dataset_options=open_dataset_options,
)
ds